In [1]:
import tensorflow as tf
import numpy as np
import unicodedata
import re
import pandas as pd

In [2]:

def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)   # it is used to convert all acented characters è to e etc2
        if unicodedata.category(c) != 'Mn')


def normalize_string(s):
    s = unicode_to_ascii(s)   # why we use \r? so that the python does not process it as a for example treat \n as enter etc but as a raw string which regex exactly needed
    s = re.sub(r'([!.?])', r' \1', s)  # the !.? is grouped, so if there is a match, the grouped will be position at 1. So using " \1" will be outputted " [?!.]"
    s = re.sub(r'[^a-zA-Z.!?]+', r' ', s)
    s = re.sub(r'\s+', r' ', s) # + means one or more repetition
    return s

In [3]:
train = pd.read_csv('train.csv')
train.head()

,id,raw_address,POI/street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung
3,3,"toko dita, kertosono",toko dita/
4,4,jl. orde baru,/jl. orde baru


In [4]:
train.columns

Index(['id', 'raw_address', 'POI/street'], dtype='object')

In [5]:
test = pd.read_csv('test.csv')
test.head()

,id,raw_address
0,0,s. par 53 sidanegara 4 cilacap tengah
1,1,"angg per, baloi indah kel. lubuk baja"
2,2,"asma laun, mand imog,"
3,3,"ud agung rej, raya nga sri wedari karanganyar"
4,4,"cut mutia, 35 baiturrahman"


In [14]:
street = train['POI/street'].apply(lambda x: x.split('/')[1])
data_train = pd.DataFrame({'raw':train['raw_address'].values,'street':street}).values
data_train[:10]
# street_split = street.apply(lambda x: x.split(' '))
# street_split

array([['jl kapuk timur delta sili iii lippo cika 11 a cicau cikarang pusat',
        'jl kapuk timur delta sili iii lippo cika'],
       ['aye, jati sampurna', ''],
       ['setu siung 119 rt 5 1 13880 cipayung', 'siung'],
       ['toko dita, kertosono', ''],
       ['jl. orde baru', 'jl. orde baru'],
       ['raya samb gede, 299 toko bb kids', 'raya samb gede'],
       ['kem mel raya, no 4 bojong rawalumbu rt 1 36 rawalumbu',
        'kem mel raya'],
       ['tela keuramat kuta alam', 'tela'],
       ['gg. i wates magersari', 'gg. i'],
       ['bunga ncole ix 2', 'bunga ncole ix']], dtype=object)

In [17]:
raw_data, target_data = list(zip(*data_train))  # remember zip will map first row to first row on each list
raw_data, target_data = list(raw_data), list(target_data)

raw_data_in = ['<start> ' + data for data in target_data]
raw_data_out = [data + ' <end>' for data in target_data]